### 本部分介绍Executor的内存分配 

#### 一. 堆外内存+堆内内存   
 Executor为了避免JVM的gc, 提供了对外内存, 直接向操作系统申请内存的方式. 这种方式需要手动申请释放内存  

#### 二. 堆内内存
1. 堆内内存四块    
 堆内内存分为4部分: Execution内存,Storage内存,用户内存和预留内存  
  1. Execution内存:(20%)   
   供tasks公用的内存.用于存放shuffle,join,排序,聚合操作的中间结果  
  2. Storage内存:(60%)  
   用于存放cache的广播变量等  
  3. 用户内存:  
   存放RDD transformation所需的信息, 如RDD的依赖信息  
  4. 预留内存:  
   固定300M. 用于存放Spark的对象
2. 内存分布图  
如下图: 整个Executor的内存由`spark.executor.memory`参数配置  
usableMemory = systemMemory - reservedMemory
<img src="img/spark-onheap.png" height="50%" width="50%">

#### 三. 堆外内存  
1. 堆外内存只有Execution和Storage内存  
2. 堆外内存默认关闭, 可以通过`spark.memory.offHeap.enabled`开启; 通过`spark.memory.offHeap.size`调整堆外内存大小


#### 四. Storage Memory 和 Execution Memory动态调整 
1. 默认情况下, 
    * **storage内存占整个Execuotr内存的60%**
    * **Execution内存占20%**, 剩下的用于保留和执行程序的堆内存.   
2. Execution和Storage内存的动态调整  
    * Spark1.5之前，Execution 内存和 Storage 内存分配是静态的:   
      如果 Execution 内存不足，即使 Storage 内存有很大空闲程序也是无法利用到的；反之亦然。这就导致我们很难进行内存的调优工作，我们必须非常清楚地了解 Execution 和 Storage 两块区域的内存分布。
    * Spark1.5 之后, Execution 内存和 Storage 内存可以互相共享的:   
      也就是说，如果 Execution 内存不足，而 Storage 内存有空闲，那么 Execution 可以从 Storage 中申请空间；**反之亦然**。所以上图中的虚线代表 Execution 内存和 Storage 内存是可以随着运作动态调整的，这样可以有效地利用内存资源。Execution 内存和 Storage 内存之间的动态调整可以概括如下：
<img src="img/storagememandexecmem.png" height="80%" width="80%">

3. 动态内存调整的具体实现逻辑如下：
    * 程序提交的时候我们都会设定基本的 Execution 内存和 Storage 内存区域（通过 spark.memory.storageFraction 参数设置）；
    * 在程序运行时，如果双方的空间都不足时，则存储到硬盘；将内存中的块存储到磁盘的策略是按照 LRU 规则进行的。若己方空间不足而对方空余时，可借用对方的空间;（存储空间不足是指不足以放下一个完整的 Block）
    * Execution 内存的空间被对方占用后，可让对方将占用的部分转存到硬盘，然后"归还"借用的空间
    * Storage 内存的空间被对方占用后，目前的实现是无法让对方"归还"，因为需要考虑 Shuffle 过程中的很多因素，实现起来较为复杂；而且 Shuffle 过程产生的文件在后面一定会被使用到，而 Cache 在内存的数据不一定在后面使用。
